# Introduccion

Este notebook contiene los procesos que se han realizado para acceder a las bases de datos del REGISTRO UNICO EMPRESARIAL para carturar información sobre las empresas (servicios y proveedores) y clientes de los contratos reportados en el sistema RENOVA de la SUPERINTENDENCIA DE VIGILANCIA Y SEGURIDAD RIVADA DE COLOMBIA. Contiene las funciones e scripts para cargar los datasets en Jupyter o Colab, a partir de archivos en formato CSV.

Los datos del RUES provienen del sistema de Datos Abiertos:

https://www.datos.gov.co/browse?Informaci%C3%B3n-de-la-Entidad_Nombre-de-la-Entidad=Superintendencia+de+Vigilancia+y+Seguridad+Privada&sortBy=newest

Esta es la primera etapa del trabajo con estos datos. 

Los objetivos de éste notebook son:
  - Capturar la información que viene de la plataforma Datos Abiertos, ya sea como archivo .CSV o como consulta SQL a la API
  - Limpiar los datos asegurando su formato, consistencia, relevancia
  - Aumentar los datos usando datasets complementarios
    - Tabla correlativa con códigos CIIU de Actividad Económica

Los datos de salida del notebook son:
  - Una tabla con los datos del RUES limpios, listos para buscar empresas por NIT. 

In [1]:
# First import the required libraries.
import pandas as pd
import numpy as np

# To load an manage files:
import io
import os

In [ ]:
# Ajustar No Maximo de Columnas para mostrar
# pd.get_option("display.max_columns")
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_columns', 20) # reset to default

In [ ]:
# Ajustar el tamaño máximo de columna que se muestra
#pd.get_option('display.max_colwidth')
#pd.set_option('display.max_colwidth', -1) 
#pd.set_option('display.max_colwidth', 50)  # reset to default

In [24]:
# Ajustar el número máximo de filas para mostrar
#pd.get_option('display.max_rows')
pd.set_option('display.max_rows', None) # mostrar todas
#pd.set_option('display.max_rows', 60) # reset to default

In [ ]:
#para poder ver los montos en valores con todas las cifras significativas en vez de notación científica
pd.set_option('display.float_format',  '{:,.2f}'.format)

# Step 1: **Cargar Archivos**

## Funciones Complementarias

#### Funcion para tener Nits de 14 caracteres

In [ ]:
nit = "00001117534465"
len(nit)

14

In [ ]:
nit = '800250741'
nit = (14-len(nit))*'0' + nit
print(nit, len(nit))

00000800250741 14


In [ ]:
a ='00000006759789' 
b = '006759789'
b in a

False

In [ ]:
b = b[:9]
b in a

True

In [162]:
# using strings of 14 characters lenght in order to match what we will get from the RUES database:
def extender_nit_string(nit):
    nit = str(nit)
    return (14-len(nit))*'0' + nit

# driver code
nit_list = ['1', '2', '3']
  
# Map method returns a map object
# so we cast it into list using list()
output_list = list(map(extender_nit_string, nit_list))
print(output_list)

['00000000000001', '00000000000002', '00000000000003']


### Funcion para calcular los dígitos de verificacion

In [3]:
# Funcion para calcular el dígito de verificación para el Registro Trubutario (RUT) o el Numero de Identificacion Tributaria (NIT)
# El valor de entrada es un número entre 5 y 15 cifras
def calcular_digito_ver(num):

    BASE = [3, 7, 13, 17, 19, 23, 29, 37, 41, 43, 47, 53, 59, 67, 71]
    DIVIDER = 11

    index = 0
    accumulator = 0
    num = str(num)
    num = [x for x in num]
    num = num[::-1]
    #print (type(num), num)
    for char in num:
        calculation = int(char) * BASE[index]
        accumulator += calculation
        index += 1

    remainder = accumulator % DIVIDER

    if (remainder>1):
        return (11-remainder)
    else:
        return remainder

In [9]:
calcular_digito_ver(800233036)

7

### Funcion para Ajustar Formato de CIIU Codes

In [81]:
def limpiar_ciuu_code(text):
    text = str(text)
    if text == 'nan':
        return np.nan
    elif len(text) == 1:
        return '0' + text + '00'
    elif len(text) == 2:
        return text + '00'
    elif len(text) < 4:
        return (4-len(text))*'0' + text
    elif len(text) > 4:
        return text[0:4]
    else:
        return text

limpiar_ciuu_code('01')
        

'0100'

## Load CSV into colab 
From Google Drive only (DO NOT RUN if in Jupyter):

In [ ]:
# For Using Google Drive, (Only if executing notebook from Google colab):
from google.colab import drive
drive.mount('/content/drive')

# after that:
# <--- Refresh mounted Drive
# <--- Look for file and get the path link

In [ ]:
# Get the current directory
print(os.getcwd())
# Check if the directory exists
print(os.path.exists('../'))

/content
True


In [ ]:
ls

drive/  sample_data/


In [ ]:
cd /content/drive/MyDrive/Observatorio SPVS/

/content/drive/MyDrive/Observatorio SPVS


## Conexión con SOCRATA (datos.gov.co)


In [ ]:
pip install sodapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 112 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
from sodapy import Socrata

In [ ]:
client = Socrata("www.datos.gov.co",
                 "TJ4yjQcX9XrhC8MB4EBqnYISj",## Cambie token_aqui por el token recien creado
                 username="ma.garzon.ra@gmail.com", ## Correo electrónico de ingreso a la plataforma de datos abiertos
                 password="Papas1597")## Contraseña de ingreso a la plataforma de datos abiertos


In [ ]:
#Test DATOS ABIERTOS RUES
%%time
socrata_dataset_identifier = "nb3d-v3n7"


#WHERE    fecha_de_firma_del_contrato >= '2018-01-01'

Query = """
SELECT * 
LIMIT 10
"""

empresas_RUES = client.get(socrata_dataset_identifier, content_type="json", query=Query)

empresas_RUES_df = pd.DataFrame(pd.DataFrame.from_dict(empresas_RUES))
empresas_RUES_df.shape

CPU times: user 30.2 ms, sys: 1.69 ms, total: 31.9 ms
Wall time: 789 ms


(10, 22)

In [ ]:
empresas_RUES_df.head()

,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,fecha_cancelacion,codigo_clase_identificacion,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matr_cula_propietario,codigo_tipo_propietario,tipo_propietario,categoria_matricula,cod_ciiu_act_econ_sec
0,02,BARRANCABERMEJA,60335,2017,20171121,TIENDA EL CENTAVO MENOS - SANTODOMINGO,03,CANCELADA,4711,20180208,01,CEDULA DE CIUDADANIA,63504995,00000063504995,6,02,BARRANCABERMEJA,60334,1,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO,NaN
1,49,OCANA,30592,2018,20180322,ORTOPEDICA TOTAL,03,CANCELADA,4773,20180906,01,CEDULA DE CIUDADANIA,63504852,00000063504852,1,49,OCANA,30591,NaN,NaN,ESTABLECIMIENTO DE COMERCIO,NaN
2,02,BARRANCABERMEJA,107073,2017,20170220,CLARIGOMAS,03,CANCELADA,4729,20170926,01,CEDULA DE CIUDADANIA,63504832,00000063504832,4,02,BARRANCABERMEJA,107072,NaN,NaN,ESTABLECIMIENTO DE COMERCIO,NaN
3,31,SAN ANDRES,21693,2022,20220512,ASADERO POLLO RICO,01,ACTIVA,5611,NaN,02,NIT,900758865,00000900758865,2,31,SAN ANDRES,35735,1,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO,NaN
4,11,CUCUTA,196896,2016,20180531,VARIEDADES NARLYN,03,CANCELADA,4719,20210426,01,CEDULA DE CIUDADANIA,63504826,00000063504826,1,11,CUCUTA,196895,1,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO,NaN


In [ ]:
#Test DATOS ABIERTOS RUES
%%time
socrata_dataset_identifier = "nb3d-v3n7"


#WHERE    fecha_de_firma_del_contrato >= '2018-01-01'

Query = """
SELECT 

camara_comercio,
razon_social,
estado_matricula,
ultimo_ano_renovado,
cod_ciiu_act_econ_pri, 
cod_ciiu_act_econ_sec, 
codigo_clase_identificacion,  	
clase_identificacion, 
numero_identificacion,
nit_propietario,
digito_verificacion

LIMIT 6000000
"""

empresas_RUES = client.get(socrata_dataset_identifier, content_type="json", query=Query)
empresas_RUES_df = pd.DataFrame(pd.DataFrame.from_dict(empresas_RUES))
empresas_RUES_df.shape

CPU times: user 39.2 s, sys: 13.8 s, total: 53 s
Wall time: 1min 51s


(5521230, 11)

### Guardar a .CSV


In [ ]:
cd 'BD RUES Empresas'

[Errno 2] No such file or directory: 'BD RUES Empresas'
/content/drive/MyDrive/Observatorio SPVS/BD RUES Empresas


In [ ]:
ls

clientes_registrados_RUES.csv
contratos_RENOVA_todos_los_clientes_con_Datos_de_RUES.csv
empresas_registradas_RUES.csv
vigilados_registrados_RUES.csv


In [ ]:
# Save the recovered data from the SQL request to the Datos Abiertos API: 
# (Uncomment to save, comment after save in order to not damage the previous file)
#empresas_RUES_df.to_csv('empresas_registradas_RUES_Socrata.csv')

## Load files from OneDrive

In [4]:
pwd

'd:\\DS4A\\SuperVigilancia\\OneDrive - Superintendencia de Vigilancia\\Proyecto Observatorio 2022\\6.Scripts\\1_Servicios_Vigilados'

In [3]:
cd "../../4.Bases_de_datos_Externas/Camaras_de_Comercio-Registro_Unico_de_Empresas_RUES"

d:\DS4A\SuperVigilancia\OneDrive - Superintendencia de Vigilancia\Proyecto Observatorio 2022\4.Bases_de_datos_Externas\Camaras_de_Comercio-Registro_Unico_de_Empresas_RUES


In [6]:

with open('./0.Raw_dataset/empresas_registradas_RUES.csv', "r", encoding='utf-8') as file:
    #read_data = file.readlines()
    #for line in read_data:
        
    empresas_RUES_df = pd.read_csv(file, sep =",", parse_dates=True, infer_datetime_format=True, encoding='utf8', dtype=str)
#close the file (not Necesary)
#f.close()
print(empresas_RUES_df.shape)
empresas_RUES_df.head(3)

(5521230, 12)


,Unnamed: 0,camara_comercio,razon_social,estado_matricula,ultimo_ano_renovado,cod_ciiu_act_econ_pri,codigo_clase_identificacion,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,cod_ciiu_act_econ_sec
0,0,BARRANCABERMEJA,TIENDA EL CENTAVO MENOS - SANTODOMINGO,CANCELADA,2017,4711,01,CEDULA DE CIUDADANIA,63504995,00000063504995,6,NaN
1,1,FLORENCIA PARA EL CAQUETA,VARIEDADES FAMI,CANCELADA,2022,4761,01,CEDULA DE CIUDADANIA,1117534465,00001117534465,9,6499
2,2,OCANA,ORTOPEDICA TOTAL,CANCELADA,2018,4773,01,CEDULA DE CIUDADANIA,63504852,00000063504852,1,NaN


In [9]:
empresas_RUES_df.drop(columns = ['Unnamed: 0'], inplace=True)

In [91]:
# Cargar la base de datos completa (Todas las Columnas) Notese que los nombres de las columnas son un poco distintos, así que hay que ajustarlas antes de continuar
empresas_RUES_df = pd.read_csv("./0.Raw_dataset/Establecimientos_-_Agencias_-_Sucursales.csv")
print(empresas_RUES_df.shape)
empresas_RUES_df.columns

C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\770387986.py:2: DtypeWarning: Columns (9,10,11,12,13,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  empresas_RUES_df = pd.read_csv("./0.Raw_dataset/Establecimientos_-_Agencias_-_Sucursales.csv")


(5548754, 24)


Index(['codigo_camara', 'camara_comercio', 'matricula', 'ultimo_ano_renovado',
       'fecha_renovacion', 'razon_social', 'codigo_estado_matricula',
       'estado_matricula', 'cod_ciiu_act_econ_pri', 'cod_ciiu_act_econ_sec',
       'ciiu3', 'ciiu4', 'fecha_cancelacion',
       'codigo_clase_identificacion_propietario',
       'clase_identificacion_propietario', 'numero_identificacion_propietario',
       'nit_propietario', 'digito_verificacion_propietario',
       'codigo_camara_propietario', 'camara_comercio_propitario',
       'matrícula_propietario', 'Codigo_tipo_propietario', 'tipo_propietario',
       'categoria_matricula'],
      dtype='object')

In [92]:
empresas_RUES_df.rename(columns = {'codigo_clase_identificacion_propietario': 'codigo_clase_identificacion',
                                   'clase_identificacion_propietario':'clase_identificacion', 
                                   'numero_identificacion_propietario':'numero_identificacion',
                                   'digito_verificacion_propietario':'digito_verificacion'}, inplace = True)

In [93]:
empresas_RUES_df["codigo_clase_identificacion"] = empresas_RUES_df["codigo_clase_identificacion"].replace([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 0.0], 
                                                    ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '00'])
#).astype(int).astype(str)

### Diccionario de Tipos de Identificacion

In [94]:
frame1 = empresas_RUES_df[["codigo_clase_identificacion", "clase_identificacion"]].value_counts().to_frame().reset_index().drop(columns=0)
frame1.set_index("codigo_clase_identificacion", inplace=True)
id_dict = frame1.to_dict()['clase_identificacion']
id_dict

{'01': 'CEDULA DE CIUDADANIA',
 '02': 'NIT',
 '06': 'SIN IDENTIFICACION',
 '03': 'CEDULA DE EXTRANJERIA',
 '05': 'PASAPORTE',
 '04': 'TARJETA DE IDENTIDAD',
 '10': 'PERMISO POR PROTECCION TEMPORAL',
 '09': 'PERMISO ESPECIAL DE PERMANENCIA',
 '08': 'DOCUMENTO EXTRANJERO',
 '07': 'REGISTRO CIVIL DE NACIMIENTO'}

# Step 2: Arreglar Formato y Consistencia de los datos

## Números de IDs en el RUES

In [98]:
empresas_RUES_df["codigo_clase_identificacion"].unique()

array(['01', '02', nan, '06', '05', '10', '03', '04', '08', '09', '07',
       '00', '0C', '0T', '0}', '0P', '0N', '0A'], dtype=object)

In [99]:
empresas_RUES_df.loc[empresas_RUES_df["codigo_clase_identificacion"].isin(['0C', '0}', '0P', '0N', '0T', '0A'])]

,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
299857,10,CARTAGO,78846,2015.0,20150331.0,AGROPECUARIA EL DOVIO F.M,3,CANCELADA,4620,4773,...,NaN,94190872,NaN,NaN,36.0,TULUA,395,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
494977,32,SANTA MARTA,254083,2022.0,20220408.0,ASECLEAN SOLUCIONES DE LIMPIEZA,1,ACTIVA,8121,8129,...,NaN,3809624,NaN,NaN,32.0,SANTA MARTA,0000000000,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
664160,32,SANTA MARTA,254953,2022.0,20220504.0,BROOKLYN BARBERIA,1,ACTIVA,9602,NaN,...,NaN,1270673,NaN,NaN,NaN,NaN,0000000000,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
975200,47,FACATATIVA,67689,2015.0,20150327.0,LAS GRANDES PROMOCIONES DE LA VEGA DE $1000 Y ...,3,CANCELADA,4719,NaN,...,NaN,80879515,00000080879515,5.0,47.0,FACATATIVA,67688,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
2256640,28,CAUCA,175219,0.0,NaN,DEEP COLLECTION,1,ACTIVA,4771,NaN,...,NaN,1001741253,00007001224532,9,28.0,CAUCA,175218,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
2271319,34,SINCELEJO,47702,0.0,NaN,SYMES BETULIA,3,CANCELADA,4773,NaN,...,NaN,8120058795,NaN,NaN,NaN,NaN,0000000000,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
3297305,30,LA GUAJIRA,144455,2018.0,20180502.0,MACHAS PIZZA,1,ACTIVA,5611,NaN,...,NaN,143525697,00000700193499,8,30.0,LA GUAJIRA,144454,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
3757333,17,IPIALES,38467,2018.0,20180502.0,GEOESPAÑA TOURS,1,ACTIVA,7990,4610,...,NaN,9108798,00000700170214,7,17.0,IPIALES,38466,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
4371459,34,SINCELEJO,47719,0.0,NaN,MOTOCOR SINCELEJO,3,CANCELADA,4541,NaN,...,NaN,8120082141,NaN,NaN,NaN,NaN,0000000000,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
4904753,32,SANTA MARTA,252589,2022.0,20220303.0,CREDITOS INVERSIONES VIVE.S,1,ACTIVA,4754,NaN,...,NaN,1451485,00000700297109,8.0,32.0,SANTA MARTA,252588,NaN,NaN,ESTABLECIMIENTO DE COMERCIO


In [100]:
# Ajustar valores de algunas celdas que tienen el gormato de cóigo de identificación mal. 
empresas_RUES_df.loc[(empresas_RUES_df["numero_identificacion"]== '94190872'), ["codigo_clase_identificacion", "clase_identificacion", "nit_propietario", "digito_verificacion"]] = ["01", "CEDULA DE CIUDADANIA", "00000094190872", "1"]

empresas_RUES_df.loc[(empresas_RUES_df["numero_identificacion"]== '80879515'), ["codigo_clase_identificacion", "clase_identificacion"]] = ["02", "NIT"]

empresas_RUES_df.loc[(empresas_RUES_df["numero_identificacion"]== '1001741253'), ["codigo_clase_identificacion", "clase_identificacion", "nit_propietario", "digito_verificacion"]] = ["01", "CEDULA DE CIUDADANIA", "00000700122453", "6"]

empresas_RUES_df.loc[empresas_RUES_df["numero_identificacion"]== '8120058795', "numero_identificacion"] = '812005879'
empresas_RUES_df.loc[(empresas_RUES_df["numero_identificacion"]== '812005879') & (empresas_RUES_df["cod_ciiu_act_econ_pri"]== '9999'), "cod_ciiu_act_econ_pri"] = "4773"
empresas_RUES_df.loc[empresas_RUES_df["numero_identificacion"]== '812005879', ["codigo_clase_identificacion", "clase_identificacion", "nit_propietario", "digito_verificacion"]] = ["02", "NIT", "00000812005879", "5"]

empresas_RUES_df.loc[empresas_RUES_df["numero_identificacion"]== '1270673', ["codigo_clase_identificacion", "clase_identificacion", "nit_propietario", "digito_verificacion"]] = ["09", "PERMISO ESPECIAL DE PERMANENCIA", "00000700337530", "9"]

empresas_RUES_df.loc[empresas_RUES_df["numero_identificacion"]== '8120082141', "numero_identificacion"] = '812008214'
empresas_RUES_df.loc[empresas_RUES_df["numero_identificacion"]== '812008214', ["cod_ciiu_act_econ_pri", "codigo_clase_identificacion", "clase_identificacion", "numero_identificacion", "nit_propietario", "digito_verificacion"]] = ["4541", "02", "NIT", "812008214", "00000812008214", "1"]

empresas_RUES_df.loc[empresas_RUES_df["numero_identificacion"]== '1451485', ["codigo_clase_identificacion", "clase_identificacion"]] = ['09', 'PERMISO ESPECIAL DE PERMANENCIA']

empresas_RUES_df.loc[empresas_RUES_df["numero_identificacion"]=='0000000000000', ["codigo_clase_identificacion", "clase_identificacion", "numero_identificacion"]] = ['06', 'SIN IDENTIFICACION', np.nan]

empresas_RUES_df.loc[empresas_RUES_df["numero_identificacion"]=='-NO-REPORTO', ["codigo_clase_identificacion", "clase_identificacion", "numero_identificacion"]] = ['06', 'SIN IDENTIFICACION', np.nan]


# Mostrar resultados
#empresas_RUES_df[empresas_RUES_df["numero_identificacion"].isin(['812008214', '1451485', '1270673', '812005879', '1001741253', '80879515', '94190872'])][["codigo_clase_identificacion", "clase_identificacion", "numero_identificacion", "nit_propietario", "digito_verificacion"]]
empresas_RUES_df.loc[empresas_RUES_df["codigo_clase_identificacion"].isin(['0C', '0}', '0P', '0N', '0T', '0A'])]
#empresas_RUES_df.loc[(~empresas_RUES_df["codigo_clase_identificacion"].isin([np.nan]))  & (empresas_RUES_df["clase_identificacion"].isin([np.nan]))]
#empresas_RUES_df.loc[(empresas_RUES_df["codigo_clase_identificacion"]!='00') & (empresas_RUES_df["numero_identificacion"]=='0000000000000'), "clase_identificacion"].unique()
#empresas_RUES_df.loc[(empresas_RUES_df["codigo_clase_identificacion"]!='00') & (empresas_RUES_df["numero_identificacion"]=='0000000000000'), "codigo_clase_identificacion"].unique()

,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
494977,32,SANTA MARTA,254083,2022.0,20220408.0,ASECLEAN SOLUCIONES DE LIMPIEZA,1,ACTIVA,8121,8129,...,NaN,3809624,NaN,NaN,32.0,SANTA MARTA,0000000000,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
3297305,30,LA GUAJIRA,144455,2018.0,20180502.0,MACHAS PIZZA,1,ACTIVA,5611,NaN,...,NaN,143525697,00000700193499,8,30.0,LA GUAJIRA,144454,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
3757333,17,IPIALES,38467,2018.0,20180502.0,GEOESPAÑA TOURS,1,ACTIVA,7990,4610,...,NaN,9108798,00000700170214,7,17.0,IPIALES,38466,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO


In [101]:
empresas_RUES_df.loc[empresas_RUES_df["codigo_clase_identificacion"].isin(['0C', '0}', '0P', '0N', '0T', '0A']), ["codigo_clase_identificacion", "clase_identificacion"]] = ['00', 'NO DEFINIDO']
# Verificar lo hecho:
#empresas_RUES_df.loc[(empresas_RUES_df["codigo_clase_identificacion"].isin(['00'])) & (~empresas_RUES_df["numero_identificacion"].isin([np.nan]))]
empresas_RUES_df.loc[(~empresas_RUES_df["codigo_clase_identificacion"].isin([np.nan]))  & (empresas_RUES_df["clase_identificacion"].isin([np.nan])), ["codigo_clase_identificacion", "clase_identificacion"]] = ['00', 'NO DEFINIDO']
empresas_RUES_df.loc[(empresas_RUES_df["codigo_clase_identificacion"].isin([np.nan])) & (~empresas_RUES_df["numero_identificacion"].isin([np.nan])) , ["codigo_clase_identificacion", "clase_identificacion"]] = ['00', 'NO DEFINIDO']
empresas_RUES_df.loc[(empresas_RUES_df["clase_identificacion"].isin([np.nan])) & (~empresas_RUES_df["numero_identificacion"].isin([np.nan])), ["codigo_clase_identificacion", "clase_identificacion"]] = ['00', 'NO DEFINIDO']

# Verificar lo hecho:
#empresas_RUES_df.loc[(empresas_RUES_df["clase_identificacion"].isin([np.nan])) & (~empresas_RUES_df["numero_identificacion"].isin([np.nan]))]
#empresas_RUES_df.loc[empresas_RUES_df["codigo_clase_identificacion"].isin([np.nan]) , ["codigo_clase_identificacion", "clase_identificacion","numero_identificacion"]].describe()
empresas_RUES_df.loc[empresas_RUES_df["codigo_clase_identificacion"].isin(['00']) , ["codigo_clase_identificacion", "clase_identificacion","numero_identificacion"]].describe()

,codigo_clase_identificacion,clase_identificacion,numero_identificacion
count,725,725,725
unique,1,1,677
top,00,NO DEFINIDO,90938032
freq,725,725,7


In [102]:
empresas_RUES_df.loc[(empresas_RUES_df["codigo_clase_identificacion"].isin(['06'])) & (~empresas_RUES_df["numero_identificacion"].isin([np.nan])), ["codigo_clase_identificacion", "clase_identificacion","numero_identificacion"]].describe()

,codigo_clase_identificacion,clase_identificacion,numero_identificacion
count,10056,10056,10056
unique,1,1,8400
top,06,SIN IDENTIFICACION,1
freq,10056,10056,794


In [103]:
empresas_RUES_df.loc[(empresas_RUES_df["codigo_clase_identificacion"].isin(['06'])) & (~empresas_RUES_df["numero_identificacion"].isin([np.nan])), ["codigo_clase_identificacion", "clase_identificacion"]] = ['00', 'NO DEFINIDO']
# Cuantos registros quedan imposibles de identificar?
empresas_RUES_df.loc[(empresas_RUES_df["codigo_clase_identificacion"].isin(['06'])) & (empresas_RUES_df["numero_identificacion"].isin([np.nan]))].shape

(273854, 24)

In [104]:
# Es 'numero_identificacion' y 'nit_propietario' lo mismo? Al parecer si
# Esta es la lista de los que son distintos. Podemos ver que 'numero_identificacion' incluye el dígito de verificacion, en algunos casos y el formato es inconsistente.
## ESTE CODIGO SE DEMORA MUCHO
#print(empresas_RUES_df[empresas_RUES_df.apply(lambda x: x['numero_identificacion'] in x['nit_propietario'], axis=1)].shape)
#print(empresas_RUES_df[~(empresas_RUES_df.apply(lambda x: x['numero_identificacion'] in x['nit_propietario'], axis=1))].shape)
#empresas_RUES_df[~(empresas_RUES_df.apply(lambda x: x['numero_identificacion'] in x['nit_propietario'], axis=1))][['numero_identificacion', 'nit_propietario']].head(20)

Cuando no se tiene NIT ni No de identificación, en teoría es posible encontrar el NIT correcto usando el registro mercantil sobre la misma tabla

Pero no tiene sentido en este momento buscar todas las instancias de cada registro.

Aqui se vé que algunos registros tuvieron un NIT en algun momento y fué borrado:

In [105]:
# Cuando no se tiene NIT ni No de identificación, en teoría es posible encontrar el NIT correcto usando el registro mercantil sobre la misma tabla
# Pero no tiene sentido en este momento buscar todas las instancias de cada registro.
# Aqui se vé que algunos registros tuvieron un NIT en algum momento y fué borrado
empresas_RUES_df.loc[(empresas_RUES_df["numero_identificacion"].isin([np.nan])) &
                     (empresas_RUES_df["nit_propietario"].isin([np.nan])) &
                     (~empresas_RUES_df["digito_verificacion"].isin([np.nan]))].head(100) #["codigo_clase_identificacion", "clase_identificacion","numero_identificacion", "nit_propietario", "digito_verificacion"]

,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
14,8,CALI,561619,2001.0,20010530.0,JUEGO DE SAPO EL CORRALITO DE GUADUA,3,CANCELADA,9311,NaN,...,SIN IDENTIFICACION,NaN,NaN,0.0,8.0,CALI,561618,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
55,10,CARTAGO,3969,1974.0,19740930.0,DROGUERIA VETERINARIA EL VAQUE,3,CANCELADA,9999,NaN,...,SIN IDENTIFICACION,NaN,NaN,0.0,0.0,RUES,0000000000,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
84,8,CALI,528134,2000.0,20000214.0,APUESTAS MAGA LIMITADA,15,MATRÍCULA CANCELADA LEY 1429,O924202,NaN,...,NIT,NaN,NaN,6.0,8.0,CALI,528133,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
97,8,CALI,267681,1999.0,19900629.0,COLOMBIANA DE ELECTRODOMESTICOS,15,MATRÍCULA CANCELADA LEY 1429,G523500,NaN,...,NIT,NaN,NaN,6.0,8.0,CALI,66665,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
108,8,CALI,616185,2005.0,20030827.0,SURTI CAMPESINO,3,CANCELADA,G522400,NaN,...,NIT,NaN,NaN,2.0,8.0,CALI,383779,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2877,8,CALI,132433,1984.0,0.0,SANCHEZ JULIO ARTURO,15,MATRÍCULA CANCELADA LEY 1429,G521102,NaN,...,NIT,NaN,NaN,6.0,8.0,CALI,132432,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
2890,8,CALI,831307,2015.0,20150320.0,MICROFINANZAS & DESARROLLO S.A.,3,CANCELADA,6499,NaN,...,SIN IDENTIFICACION,NaN,NaN,0.0,8.0,CALI,825977,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
2904,3,BARRANQUILLA,556540,2012.0,20121031.0,ASOCIACION DE SOLUCIONES TECNICAS S.A.S.,1,ACTIVA,4290,NaN,...,NIT,NaN,NaN,1.0,3.0,BARRANQUILLA,556539,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
2921,8,CALI,942556,2016.0,20160106.0,EL CHONTADURO,3,CANCELADA,4724,NaN,...,SIN IDENTIFICACION,NaN,NaN,0.0,8.0,CALI,942555,NaN,NaN,ESTABLECIMIENTO DE COMERCIO


Eliminar registros que son imposibles de identificar con NIT y No de identificacion:

In [222]:
 # Eliminar registros que son imposibles de identificar con NIT y No de identificacion:
empresas_RUES_df2 = empresas_RUES_df.drop(empresas_RUES_df[(empresas_RUES_df["numero_identificacion"].isin([np.nan])) & (empresas_RUES_df["nit_propietario"].isin([np.nan]))].index)
print(empresas_RUES_df2.shape)
empresas_RUES_df2.isna().sum()

(4783939, 24)


codigo_camara                        0
camara_comercio                      0
matricula                            0
ultimo_ano_renovado                  0
fecha_renovacion                992313
razon_social                        17
codigo_estado_matricula              0
estado_matricula                     0
cod_ciiu_act_econ_pri            24946
cod_ciiu_act_econ_sec          3187415
ciiu3                          4068556
ciiu4                          4459915
fecha_cancelacion              2100043
codigo_clase_identificacion          0
clase_identificacion                 0
numero_identificacion              359
nit_propietario                2091416
digito_verificacion            1051667
codigo_camara_propietario       153904
camara_comercio_propitario      153951
matrícula_propietario           122069
Codigo_tipo_propietario        2276733
tipo_propietario               2509651
categoria_matricula                  0
dtype: int64

A partir de éste punto se podría realizar una búsqueda y un ajuste de los NITS y números de identificación que no cumplen con el formato numérico
Pero encontramos que la cantidad de datos que tienen este problema son solo 80 + 211 + 237 filas. Entonces también se procede a eliminarlas

In [223]:
empresas_RUES_df2[(empresas_RUES_df2['nit_propietario'].isin(['0000EN0TRAMITE', '00000EN0TRAMIT'])) &
                  (empresas_RUES_df2["numero_identificacion"].isin(['EN0TRAMIT', 'EN0TRAMITE']))]

,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
305496,44,URABA,14219,0.0,NaN,CALZA RUSO,3,CANCELADA,9999,NaN,...,NIT,EN0TRAMIT,00000EN0TRAMIT,E,44.0,URABA,14218,3.0,COPROPIETARIO,ESTABLECIMIENTO DE COMERCIO
3546360,44,URABA,14574,0.0,NaN,CALZADO SHALOOM,3,CANCELADA,9999,NaN,...,NIT,EN0TRAMIT,00000EN0TRAMIT,E,44.0,URABA,14218,3.0,COPROPIETARIO,ESTABLECIMIENTO DE COMERCIO


In [224]:
empresas_RUES_df2[(empresas_RUES_df2['nit_propietario'].str.contains('[A-Za-z]')) &
                  (empresas_RUES_df2["numero_identificacion"].str.contains('[A-Za-z]'))]

,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
89109,3,BARRANQUILLA,780513,2021.0,20210201.0,OPTICA FUSEMA,1,ACTIVA,4755,NaN,...,NO DEFINIDO,HN339397,000000HN339397,NaN,3.0,BARRANQUILLA,780512,0.0,NaN,ESTABLECIMIENTO DE COMERCIO
94966,27,PEREIRA,14429102,0.0,NaN,PROVISEC,3,CANCELADA,9999,NaN,...,NIT,NO REPORTA,0000NO REPORTA,NaN,21.0,MEDELLIN PARA ANTIOQUIA,24438903,NaN,NaN,AGENCIA
181144,19,MAGANGUE,12148,0.0,NaN,ASOCIACION MUTUALISTA DE BOLIVAR,3,CANCELADA,9999,NaN,...,NIT,CALLE0DE0LAS0D,CALLE0DE0LAS0D,A,19.0,MAGANGUE,9000500008,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
187191,30,LA GUAJIRA,12679,0.0,NaN,CORDELL REPRESENTACIONES,3,CANCELADA,9999,NaN,...,NIT,NO REPORTO,0000NO REPORTO,NaN,3.0,BARRANQUILLA,94495,NaN,NaN,SUCURSAL
235846,43,SEVILLA,19212,2022.0,20220713.0,KEBAB DEL VALLE,1,ACTIVA,5611,NaN,...,PASAPORTE,AA0231732,00000AA0231732,4,43.0,SEVILLA,19210,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5390976,30,LA GUAJIRA,171320,2022.0,20220614.0,MANGO BICHE,1,ACTIVA,4771,NaN,...,PASAPORTE,TD006005,000000TD006005,4.0,30.0,LA GUAJIRA,171319,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
5486648,17,IPIALES,54372,2022.0,20220919.0,CARMEN MIREYA GOYES BURBANO,1,ACTIVA,4631,4641,...,PASAPORTE,A8210096,000000A8210096,7.0,17.0,IPIALES,54371,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
5488189,26,PASTO,222888,2022.0,20220824.0,GASTRO BAR EL PADRINO,1,ACTIVA,5611,5630,...,PASAPORTE,A4241229,000000A4241229,8.0,26.0,PASTO,222887,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
5503278,47,FACATATIVA,168356,2022.0,20220916.0,PATITAS PET STORE FACA,1,ACTIVA,4759,NaN,...,PASAPORTE,F36014230,00000F36014230,1.0,47.0,FACATATIVA,168355,NaN,NaN,ESTABLECIMIENTO DE COMERCIO


In [225]:
empresas_RUES_df2[(empresas_RUES_df2['nit_propietario'].str.contains('[A-Za-z]')) &
                  (~empresas_RUES_df2["numero_identificacion"].str.contains('[A-Za-z]', na=False))]

,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
7196,44,URABA,20338,2003.0,20031215.0,CONFECCIONES LA FIESTA,3,CANCELADA,9999,NaN,...,CEDULA DE CIUDADANIA,8333301,0000EN0TRAMITE,0.0,44.0,URABA,643,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
13103,27,PEREIRA,4218002,0.0,NaN,TIENDA FANNY,3,CANCELADA,9999,NaN,...,CEDULA DE CIUDADANIA,42075346,00000NOREPORTA,0.0,27.0,PEREIRA,4217901,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
39544,50,CASANARE,165963,2022.0,20220722.0,PARADOR EL PUNTO DEL SABOR,1,ACTIVA,1081,5613,...,CEDULA DE CIUDADANIA,70908906,00000000NOTIEN,E,50.0,CASANARE,165962,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
47348,27,PEREIRA,15237502,0.0,NaN,FANTASY FOUNTAIN CHOCOLATE,3,CANCELADA,9999,NaN,...,CEDULA DE CIUDADANIA,10087631,000000000000NR,0,27.0,PEREIRA,11596801,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
70024,51,ORIENTE ANTIOQUENO,26305,2016.0,20160323.0,HELADERIA EL RINCON DE MEXICO,3,CANCELADA,5630,NaN,...,CEDULA DE CIUDADANIA,3495845,000000entramit,e,51.0,ORIENTE ANTIOQUENO,29052,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5255643,44,URABA,26596,2017.0,20170329.0,TIENDA FLOR ALVA,3,CANCELADA,4711,4722,...,CEDULA DE CIUDADANIA,39417725,00000EN0TRAMIT,E,44.0,URABA,26595,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
5262001,51,ORIENTE ANTIOQUENO,37164,2017.0,20171017.0,PARQUEO LA 47,3,CANCELADA,5221,NaN,...,CEDULA DE CIUDADANIA,20531847,000000entramit,e,51.0,ORIENTE ANTIOQUENO,37163,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
5264456,40,VILLAVICENCIO,90131,2017.0,20170224.0,SUPERTIENDA LOS TERECOS,3,CANCELADA,4711,NaN,...,CEDULA DE CIUDADANIA,40384977,00000NO0REPORT,A,40.0,VILLAVICENCIO,90130,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
5516242,32,SANTA MARTA,260033,2022.0,20220902.0,TALLER Y METALICAS LA PAZ,1,ACTIVA,4390,NaN,...,CEDULA DE CIUDADANIA,7152848,nestorcastille,j,32.0,SANTA MARTA,260032,NaN,NaN,ESTABLECIMIENTO DE COMERCIO


In [226]:
empresas_RUES_df2[(~empresas_RUES_df2['nit_propietario'].str.contains('[A-Za-z]', na=False)) &
                  (empresas_RUES_df2["numero_identificacion"].str.contains('[A-Za-z]'))]

,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
19731,4,BOGOTA,1809211,2017.0,20170331.0,B ESTILO,1,ACTIVA,4652,4771,...,CEDULA DE EXTRANJERIA,E358439,NaN,NaN,4.0,BOGOTA,1809210,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
36004,4,BOGOTA,2302360,2014.0,20140307.0,MUNDO DE PIJAMAS,3,CANCELADA,4771,NaN,...,CEDULA DE EXTRANJERIA,E317888,NaN,8,4.0,BOGOTA,1957627,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
40020,27,PEREIRA,17166702,2013.0,20140127.0,LA PASTERIA ITALIANA,3,CANCELADA,5612,NaN,...,CEDULA DE EXTRANJERIA,E396216,NaN,NaN,27.0,PEREIRA,17795901,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
45852,32,SANTA MARTA,242594,2021.0,20210609.0,MOMENTUM EXPERIENCE,1,ACTIVA,8552,NaN,...,PASAPORTE,12CT51463,00000700271502,7,32.0,SANTA MARTA,242593,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
57557,11,CUCUTA,208782,0.0,NaN,REPRESENTACIONES OÑO,3,CANCELADA,7310,4719,...,CEDULA DE EXTRANJERIA,E395638,00007000874883,3,11.0,CUCUTA,208781,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5498061,46,PUTUMAYO,88474,2022.0,20220926.0,MULHER BONITA PELUQUERIA Y DISTRIBUIDORA,1,ACTIVA,4773,9602,...,PASAPORTE,A4406690,00000700344382,4.0,46.0,PUTUMAYO,88473,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
5502965,4,BOGOTA,3586718,0.0,NaN,SANTA ANA GASTRO BAR,1,ACTIVA,1084,5611,...,PASAPORTE,PAN977977,NaN,NaN,4.0,BOGOTA,3586716,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
5539072,55,ABURRA SUR,255074,2022.0,20220829.0,RESTAURANTE DRAGON S,1,ACTIVA,5611,NaN,...,PASAPORTE,EII874038,00000700324717,2,55.0,ABURRA SUR,255073,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
5543218,27,PEREIRA,18189882,2022.0,20220829.0,SOEKOEIMPEX,1,ACTIVA,4620,4631.0,...,PASAPORTE,NT8655C60,00000700280708,5,27.0,PEREIRA,18189881,NaN,NaN,ESTABLECIMIENTO DE COMERCIO


## Números de NITs en el RUES

Verificar que tienen el valor correcto

In [227]:
# Se puede intentar corregir algunos números, uno por uno:
empresas_RUES_df2.loc[empresas_RUES_df2["numero_identificacion"] =='41511407-', ["numero_identificacion", "nit_propietario", "digito_verificacion"]] = ["41511407", "00000041511407", calcular_digito_ver(41511407)] 
empresas_RUES_df2.loc[empresas_RUES_df2["numero_identificacion"] =='28681158-', ["numero_identificacion", "nit_propietario", "digito_verificacion"]] = ["28681158", "00000028681158", calcular_digito_ver(28681158)]


In [228]:
empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"] =='0000800025201A', "nit_propietario"] = "00000800025201" 
empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"] =='0000090703017A', "nit_propietario"] = "00000090703017"
empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"] =='0000060020100A', "nit_propietario"] = "00000060020100" 
empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"] =='0000090908014A', "nit_propietario"] = "00000090908014"
empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"] =='0000890702136A', "nit_propietario"] = "00000890702136"
empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"] =='0008300510214C', "nit_propietario"] = "00008300510214" 
empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"] =='0008300968403C', "nit_propietario"] = "00008300968403"
empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"] =='00001O75668091', "nit_propietario"] = "00001075668091"
empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"] =='S00000028831043', "nit_propietario"] = "00000028831043"
empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"] =='S00000033979539', "nit_propietario"] = "00000033979539"
empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"] =='S00000882824201', "nit_propietario"] = "00000882824201"
empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"] =='D00000466805732', "nit_propietario"] = "00000466805732"
empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"] =='S00000028831043', "nit_propietario"] = "00000028831043"
empresas_RUES_df2.loc[empresas_RUES_df2["numero_identificacion"] =='33806908', "nit_propietario"] = "00000033806908"


In [229]:
#Eliminar filas que no tienen numeros para identificacion
#empresas_RUES_df2[~empresas_RUES_df2['numero_identificacion'].str.contains('[0-9]', na=False)]
 # En vez de usar drop() se usó seleccion. El error: "Cannot mask with non-boolean array containing NA / NaN values" se solucionó usando , na=False
empresas_RUES_df2 = empresas_RUES_df2.loc[(empresas_RUES_df2['numero_identificacion'].str.contains('[0-9]', na=False))] 
#empresas_RUES_df2[~empresas_RUES_df2['numero_identificacion'].str.contains('[0-9]', na=False)].head()
print(empresas_RUES_df2.shape)

(4783510, 24)


In [230]:
# Eliminar los NITs que tienen caracteres alfabéticos en la identificación. estos son palabras y son solo ~200 casos  que son pasaportes o no reportan)
#empresas_RUES_df2 = empresas_RUES_df2.drop(empresas_RUES_df2.loc[empresas_RUES_df2['nit_propietario'].str.contains('[A-Za-z]', na=False)].index)
empresas_RUES_df2 = empresas_RUES_df2.loc[(~empresas_RUES_df2['nit_propietario'].str.contains('[A-Za-z]', na=False))]
print(empresas_RUES_df2.shape)

(4783254, 24)


In [231]:
empresas_RUES_df2 = empresas_RUES_df2.drop(empresas_RUES_df2.loc[empresas_RUES_df2['nit_propietario'].str.contains('[A-Za-z]', na=False)].index)
print(empresas_RUES_df2.shape)

(4783254, 24)


In [232]:
# Se pueden eliminar filas específicas con problemas (en el dataframe filtrado. Si es el dataframe original estos índicas son distintos y se elimina lo que no es)
#empresas_RUES_df2.drop([94162], inplace = True)
#empresas_RUES_df2.drop([718936, 769498, 930182, 1515068, 1977579, 3993817, 4072552], inplace = True)

### Asignar el NIT a partir del No de ID


In [233]:
#Visualizar:
#empresas_RUES_df2[empresas_RUES_df2['numero_identificacion'].str.contains(' - ', na=False)]
#empresas_RUES_df2[empresas_RUES_df2['numero_identificacion'].str.contains('=', na=False)]
#empresas_RUES_df2[empresas_RUES_df2['numero_identificacion'].str.contains('.', na=False)]

In [234]:
empresas_RUES_df2['numero_identificacion'] = empresas_RUES_df2['numero_identificacion'].str.replace(' - ', '-')
empresas_RUES_df2['numero_identificacion'] = empresas_RUES_df2['numero_identificacion'].str.replace('=', '-')
empresas_RUES_df2['numero_identificacion'] = empresas_RUES_df2['numero_identificacion'].str.replace('.', '')
empresas_RUES_df2['numero_identificacion'] = empresas_RUES_df2['numero_identificacion'].str.replace(' ', '-')

C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\332952719.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresas_RUES_df2['numero_identificacion'] = empresas_RUES_df2['numero_identificacion'].str.replace('.', '')


In [235]:
empresas_RUES_df3 = empresas_RUES_df2[(empresas_RUES_df2['codigo_clase_identificacion'] == '02')&(empresas_RUES_df2['nit_propietario'].isin([np.nan]))].copy()
print(empresas_RUES_df3.shape)
empresas_RUES_df3 = empresas_RUES_df3.drop(empresas_RUES_df3[empresas_RUES_df3['numero_identificacion'].str.contains('[A-Za-z]', na=False)].index)
print(empresas_RUES_df3.shape)
# Verificar:
#print(empresas_RUES_df3[~empresas_RUES_df['nit_propietario'].isna()].shape)
#print(empresas_RUES_df3[empresas_RUES_df2['numero_identificacion'].isna()].shape)

(784392, 24)
(784389, 24)


In [236]:
empresas_RUES_df4 = empresas_RUES_df3.loc[empresas_RUES_df3['numero_identificacion'].str.contains('-', na=False)]
print(empresas_RUES_df4.shape)
empresas_RUES_df4.head()

(356, 24)


,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
280,16,IBAGUE,178725,2008.0,20080401.0,SERVICIOS POSTALES NACIONALES,3,CANCELADA,9999,NaN,...,NIT,900062917-9,NaN,NaN,NaN,NaN,0000000000,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
10972,16,IBAGUE,167541,2008.0,20080314.0,KATIOS TIPICO - PARRILLA,3,CANCELADA,9999,NaN,...,NIT,830055803-5,NaN,NaN,NaN,NaN,0000000000,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
24290,52,MAGDALENA MEDIO,46385,2012.0,20120112.0,GIH PUERTO TRIUNFO IPS,3,CANCELADA,8621,NaN,...,NIT,830102106-1,NaN,NaN,4.0,BOGOTA,11794,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
53155,22,MONTERIA,91594,2011.0,20110420.0,SUPER POWER,3,CANCELADA,9999,NaN,...,NIT,900159468-1,NaN,NaN,NaN,NaN,0000000000,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
56551,16,IBAGUE,173015,2010.0,20100405.0,RAZA IBAGUE,3,CANCELADA,9999,NaN,...,NIT,860019205-7,NaN,NaN,NaN,NaN,0000000000,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO


In [237]:
#Asignar valores de Dígitos de Verificación y de NITs a las columnas correspondientes
# Esta línea sólo se debe correr una vez. de otro modo se alteran todos los números
#empresas_RUES_df4["numero_identificacion"].str.strip("")
empresas_RUES_df4['digito_verificacion'] = empresas_RUES_df4["numero_identificacion"].str.strip().str[-1]
empresas_RUES_df4['numero_identificacion'] = empresas_RUES_df4["numero_identificacion"].str.strip().str[:-2]
empresas_RUES_df4['nit_propietario'] = empresas_RUES_df4["numero_identificacion"].apply(lambda x : extender_nit_string(x))
# Actualizar Valores en el DataFrame Principal
empresas_RUES_df2.loc[empresas_RUES_df4.index] = empresas_RUES_df4
empresas_RUES_df4.head()


C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\4244518606.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_RUES_df4['digito_verificacion'] = empresas_RUES_df4["numero_identificacion"].str.strip().str[-1]
C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\4244518606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_RUES_df4['numero_identificacion'] = empresas_RUES_df4["numero_identificacion"].str.strip().str[:-2]
C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\4244518606.py:6: Setting

,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
280,16,IBAGUE,178725,2008.0,20080401.0,SERVICIOS POSTALES NACIONALES,3,CANCELADA,9999,NaN,...,NIT,900062917,00000900062917,9,NaN,NaN,0000000000,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
10972,16,IBAGUE,167541,2008.0,20080314.0,KATIOS TIPICO - PARRILLA,3,CANCELADA,9999,NaN,...,NIT,830055803,00000830055803,5,NaN,NaN,0000000000,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
24290,52,MAGDALENA MEDIO,46385,2012.0,20120112.0,GIH PUERTO TRIUNFO IPS,3,CANCELADA,8621,NaN,...,NIT,830102106,00000830102106,1,4.0,BOGOTA,11794,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
53155,22,MONTERIA,91594,2011.0,20110420.0,SUPER POWER,3,CANCELADA,9999,NaN,...,NIT,900159468,00000900159468,1,NaN,NaN,0000000000,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
56551,16,IBAGUE,173015,2010.0,20100405.0,RAZA IBAGUE,3,CANCELADA,9999,NaN,...,NIT,860019205,00000860019205,7,NaN,NaN,0000000000,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO


In [238]:
empresas_RUES_df3 = empresas_RUES_df3.loc[~empresas_RUES_df3['numero_identificacion'].str.contains('-', na=False)]
print(empresas_RUES_df3.shape)
empresas_RUES_df3.head()

(784033, 24)


,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
1,5,BUCARAMANGA,9000177225,2022.0,20220425.0,TIENDA ANGELITOS CON AMOR,1,ACTIVA,4759,4771,...,NIT,37896777,NaN,3.0,5.0,BUCARAMANGA,177224,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
17,5,BUCARAMANGA,9000167235,2019.0,20190222.0,HOSTEL SAN GIL,3,CANCELADA,5519,NaN,...,NIT,41619240,NaN,9.0,5.0,BUCARAMANGA,167233,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
29,5,BUCARAMANGA,9000019410,2022.0,20220209.0,LABORATORIO OPTICO EL CRISTAL,1,ACTIVA,2670,4774,...,NIT,37822564,NaN,4.0,5.0,BUCARAMANGA,52454,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
37,5,BUCARAMANGA,9000407812,0.0,NaN,ORGANIZACIÓN GARCIA & PEÑALOZA,3,CANCELADA,9602,5221,...,NIT,901194230,NaN,8.0,5.0,BUCARAMANGA,407811,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
53,45,SUR Y ORIENTE DEL TOLIMA,78352,2014.0,20140313.0,PROTECCION Y VIDA,3,CANCELADA,6621,NaN,...,NIT,900702353,NaN,2.0,NaN,NaN,0000000000,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO


In [239]:
# Verificar que algunos datos tienen bien el dígito de verificacion
calcular_digito_ver(10892394)

1

In [240]:
# Asignar el NIT de acuerdo al Número de Identificación existente, que está marcado como un NIT y que coincide con el cálculo del dígito de Verificación.
empresas_RUES_df3['nit_propietario'] = empresas_RUES_df3["numero_identificacion"].apply(lambda x : extender_nit_string(x))
# Actualizar Valores en el DataFrame Principal
empresas_RUES_df2.loc[empresas_RUES_df3.index] = empresas_RUES_df3

In [241]:
# Corregir el formato de los No de identificación  cuando tienen "-" al final del string.
empresas_RUES_df5 = empresas_RUES_df2[empresas_RUES_df2["numero_identificacion"].str.strip().str[-1]=='-']
print(empresas_RUES_df5.shape)

(129, 24)


In [242]:
empresas_RUES_df5['nit_propietario'] = empresas_RUES_df5['nit_propietario'].str.strip().str[:-1]
empresas_RUES_df5['numero_identificacion'] = empresas_RUES_df5["numero_identificacion"].str.strip().str[:-1]
# Actualizar Valores en el DataFrame Principal
empresas_RUES_df2.loc[empresas_RUES_df5.index] = empresas_RUES_df5

C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\1541504340.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_RUES_df5['nit_propietario'] = empresas_RUES_df5['nit_propietario'].str.strip().str[:-1]
C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\1541504340.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_RUES_df5['numero_identificacion'] = empresas_RUES_df5["numero_identificacion"].str.strip().str[:-1]


In [243]:
# Corregir el formato de los No de identificación  cuando tienen "-" al final del string.
empresas_RUES_df6 = empresas_RUES_df2[(empresas_RUES_df2["numero_identificacion"].str.strip().str[-2]=='-')&
                                      (empresas_RUES_df2["numero_identificacion"].str.strip().str[:-3].str.isnumeric())]
print(empresas_RUES_df6.shape)
empresas_RUES_df6.head()

(1342, 24)


,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
4350,51,ORIENTE ANTIOQUENO,135058,2021.0,20210315.0,MUNDOLUX S.A.S- SEDE GUARNE,1,ACTIVA,4759,NaN,...,NIT,900363032-8,00000900363032,8.0,21.0,MEDELLIN PARA ANTIOQUIA,2143235412,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
5023,22,MONTERIA,65004,2020.0,20200423.0,PLASTICAUCA,1,ACTIVA,4690,4719,...,NIT,900995087-4,00000900995087,4.0,21.0,MEDELLIN PARA ANTIOQUIA,567041,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
28470,11,CUCUTA,211367,2015.0,20150609.0,AGENCIA DE TEXTILES Y CONFECCIONES EXPOTELA CU...,3,CANCELADA,4751,4771,...,NIT,860400157-4,000860400157-4,9.0,4.0,BOGOTA,148257,NaN,NaN,AGENCIA
30183,38,TUNJA,95791,2018.0,20190116.0,EDS EL PORTAL DE TUNJA,3,CANCELADA,4661,NaN,...,NIT,830031824-6,00000830031824,6.0,47.0,FACATATIVA,18868,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
36834,32,SANTA MARTA,233435,2022.0,20220725.0,GRUPO EMPRESARIAL DAJE PIJIÑO DEL CARMEN,1,ACTIVA,4923,4290,...,NIT,901392790-1,00000901392790,1,0.0,RUES,0000000000,NaN,NaN,ESTABLECIMIENTO DE COMERCIO


In [244]:
#Asignar valores de Dígitos de Verificación y de NITs a las columnas correspondientes
# Esta línea sólo se debe correr una vez. de otro modo se alteran todos los números
#empresas_RUES_df4["numero_identificacion"].str.strip("")
empresas_RUES_df6['digito_verificacion'] = empresas_RUES_df6["numero_identificacion"].str.strip().str[-1]
empresas_RUES_df6['numero_identificacion'] = empresas_RUES_df6["numero_identificacion"].str.strip().str[:-2]
empresas_RUES_df6['nit_propietario'] = empresas_RUES_df6["numero_identificacion"].apply(lambda x : extender_nit_string(x))

C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\1161732194.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_RUES_df6['digito_verificacion'] = empresas_RUES_df6["numero_identificacion"].str.strip().str[-1]
C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\1161732194.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_RUES_df6['numero_identificacion'] = empresas_RUES_df6["numero_identificacion"].str.strip().str[:-2]
C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\1161732194.py:6: Setting

In [245]:
empresas_RUES_df7 = empresas_RUES_df6[empresas_RUES_df6['nit_propietario'].str.contains("-")]
empresas_RUES_df6 = empresas_RUES_df6[~empresas_RUES_df6['nit_propietario'].str.contains("-")]
empresas_RUES_df7['nit_propietario'] = empresas_RUES_df7['nit_propietario'].str.strip().str[:-1]
empresas_RUES_df7['numero_identificacion'] = empresas_RUES_df7["numero_identificacion"].str.strip().str[:-1]

In [246]:
# Actualizar Valores en el DataFrame Principal
empresas_RUES_df2.loc[empresas_RUES_df6.index] = empresas_RUES_df6
empresas_RUES_df2.loc[empresas_RUES_df7.index] = empresas_RUES_df7
#empresas_RUES_df6.head()

In [247]:
empresas_RUES_df8 = empresas_RUES_df2[empresas_RUES_df2["nit_propietario"].str.contains("-", na=False)]
print(empresas_RUES_df8.shape)
empresas_RUES_df8.head()

(688, 24)


,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
12505,16,IBAGUE,141946,2012.0,20120113.0,CENTRO SOCIAL LIMONAR,3,CANCELADA,9999,NaN,...,CEDULA DE CIUDADANIA,38260626,000038260626-3,9.0,16.0,IBAGUE,141945,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
16995,20,MANIZALES,65464,2002.0,20020321.0,ESTANQUILLO TAURO,3,CANCELADA,9999,NaN,...,CEDULA DE CIUDADANIA,24319577,000024319577-6,5.0,20.0,MANIZALES,65808,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
30469,44,URABA,10779,0.0,NaN,CHISPI,3,CANCELADA,9999,NaN,...,CEDULA DE CIUDADANIA,8414607,000008414607-7,1.0,44.0,URABA,10778,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
40902,55,ABURRA SUR,48719,2013.0,20140212.0,RESTAURANTE LA FORTUNITA,3,CANCELADA,5611,NaN,...,CEDULA DE CIUDADANIA,21439014,000021439014-5,8,55.0,ABURRA SUR,48718,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
57789,27,PEREIRA,9844702,0.0,NaN,COMIDAS RAPIDAS LA 5TA.,3,CANCELADA,9999,NaN,...,CEDULA DE CIUDADANIA,10061527,000010061527-7,6,27.0,PEREIRA,9673601,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO


In [248]:
empresas_RUES_df8['digito_verificacion'] = empresas_RUES_df8['nit_propietario'].str.strip().str[-1]
empresas_RUES_df8['nit_propietario'] = empresas_RUES_df8['nit_propietario'].str.strip().str[:-2]
empresas_RUES_df2.loc[empresas_RUES_df8.index] = empresas_RUES_df8

C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\2672710332.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_RUES_df8['digito_verificacion'] = empresas_RUES_df8['nit_propietario'].str.strip().str[-1]
C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\2672710332.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_RUES_df8['nit_propietario'] = empresas_RUES_df8['nit_propietario'].str.strip().str[:-2]


In [249]:
empresas_RUES_df2[empresas_RUES_df2["nit_propietario"].str.contains("-", na=False)]
#empresas_RUES_df2[(~empresas_RUES_df2["nit_propietario"].str.isnumeric())].shape

,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
3635430,22,MONTERIA,32260,2014.0,20140318.0,MER-K-CENTRO,3,CANCELADA,4711,4773,...,CEDULA DE CIUDADANIA,10892394,00000000000-,0,22.0,MONTERIA,115409,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO


In [250]:
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('-', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('}', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('.', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('/', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('*', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('+', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('Ç', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace(',', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('?', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace(':', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('_', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace(' ', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('~', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('º', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('|', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('001088335749\\xad', '001088335749')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('{', '')
empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('´', '')
# Se podría haber usado   .str.extract('(\d+)', expand=False)  ??

C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\1151995661.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('}', '')
C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\1151995661.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresas_RUES_df2['nit_propietario'] = empresas_RUES_df2['nit_propietario'].str.replace('.', '')
C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\1151995661.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True

In [251]:
empresas_RUES_df2.loc[empresas_RUES_df2["numero_identificacion"]=='0000000000000', ["codigo_clase_identificacion", "clase_identificacion", "numero_identificacion"]] = ['06', 'SIN IDENTIFICACION', np.nan]
empresas_RUES_df2['nit_propietario'].head()

0    00001075287105
1    00000037896777
2    00000860034313
3               NaN
4               NaN
Name: nit_propietario, dtype: object

In [252]:
series = empresas_RUES_df2['nit_propietario'].copy()
series.astype(float)
series.astype(float).describe()

count    3.117293e+06
mean     2.229001e+10
std      3.633272e+12
min      0.000000e+00
25%      4.452260e+07
50%      4.212752e+08
75%      9.011648e+08
max      1.000000e+15
Name: nit_propietario, dtype: float64

In [253]:
empresas_RUES_df2.loc[(empresas_RUES_df2['nit_propietario'].str.len()>14) & (~empresas_RUES_df2['nit_propietario'].isna()) & (~empresas_RUES_df2['codigo_clase_identificacion'].isin(['09', '01']))]

,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
130970,10,CARTAGO,44811,1985.0,19850820.0,AGROPECUARIA TRUJILLO Y TRUJILLO LIMITADA,3,CANCELADA,9999,NaN,...,NIT,10000000000000,100000000000000,6.0,10.0,CARTAGO,10478,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
203332,55,ABURRA SUR,88025,2008.0,20080303.0,LATIN DATING AND MARRIAGE AGENCY,3,CANCELADA,9999,NaN,...,PASAPORTE,203208059,900006110016673,0,55.0,ABURRA SUR,88024,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
259409,11,CUCUTA,163637,2015.0,20151104.0,NORCERAMICA S.A.,3,CANCELADA,0811,2392,...,NIT,30000900160927,300009001609272,0,11.0,CUCUTA,163534,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
967377,38,TUNJA,1143,1975.0,19751204.0,SANCHEZ Y CIA. LTDA. ASESORES DE SEGUROS,3,CANCELADA,9999,NaN,...,NIT,91690100000000,916901000000000,4.0,38.0,TUNJA,1142,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
1687839,22,MONTERIA,65717,2012.0,20120202.0,JARDIN INFANTIL AVENTURA DE CRECER,3,CANCELADA,9999,NaN,...,NIT,30000812006540,300008120065409,7,22.0,MONTERIA,65716,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
1906437,33,SANTA ROSA DE CABAL,26377,2006.0,20060605.0,MADERAS ADRIANA,3,CANCELADA,9999,NaN,...,NIT,60000900089386,600009000893865,6.0,33.0,SANTA ROSA DE CABAL,26376,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
2121675,18,LA DORADA,12926,1994.0,19940314.0,TRANSPORTES RODRIGUEZ MAHECHA,3,CANCELADA,9999,NaN,...,NIT,56849900168998,568499001689981,5.0,18.0,LA DORADA,12925,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
2430795,56,PIEDEMONTE ARAUCANO,153,1993.0,19930224.0,EMPRESA DE TRANSPORTE URBANO DEL MUNICIPIO DE ...,3,CANCELADA,9999,NaN,...,NIT,50790173600000,507901736000000,1,56.0,PIEDEMONTE ARAUCANO,152,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
2762383,10,CARTAGO,10412,1985.0,19850729.0,AGENCIA DE SEGUROS CARLOS A.MEZA A. & CIA.LTDA.,3,CANCELADA,9999,NaN,...,NIT,10000000000000,100000000000000,6.0,10.0,CARTAGO,10411,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
2872727,57,SAN JOSE,5265,2002.0,20020916.0,NO ASIGNADO,5,NO ASIGNADO,9999,NaN,...,NIT,50000000000526,500000000005264,8,57.0,SAN JOSE,5264,1.0,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO


In [254]:
# Verificar que todos los números son correctos: (esto altera los NITS, no correrlo)
#empresas_RUES_df2['nit_propietario'].astype(float)
#empresas_RUES_df2['nit_propietario'].astype(str)
#empresas_RUES_df2['nit_propietario'].astype(float).describe()
# WARNING: no hacer lo siguitne, porque modifica el dataframe
#empresas_RUES_df2['nit_propietario'] = pd.to_numeric(empresas_RUES_df2['nit_propietario'])

# Finalmente ajustar el formato:
empresas_RUES_df2.loc[empresas_RUES_df2['nit_propietario'].str.len()<14, 'nit_propietario'] = empresas_RUES_df2.loc[empresas_RUES_df2['nit_propietario'].str.len() <14, 'nit_propietario'].apply(lambda x : extender_nit_string(x))
#empresas_RUES_df2['nit_propietario'].head()


In [153]:
empresas_RUES_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4783254 entries, 0 to 4783253
Data columns (total 25 columns):
 #   Column                       Dtype 
---  ------                       ----- 
 0   Unnamed: 0                   object
 1   codigo_camara                object
 2   camara_comercio              object
 3   matricula                    object
 4   ultimo_ano_renovado          object
 5   fecha_renovacion             object
 6   razon_social                 object
 7   codigo_estado_matricula      object
 8   estado_matricula             object
 9   cod_ciiu_act_econ_pri        object
 10  cod_ciiu_act_econ_sec        object
 11  ciiu3                        object
 12  ciiu4                        object
 13  fecha_cancelacion            object
 14  codigo_clase_identificacion  object
 15  clase_identificacion         object
 16  numero_identificacion        object
 17  nit_propietario              object
 18  digito_verificacion          object
 19  codigo_camara_propiet

In [258]:
for column in [ 'codigo_camara', 'matricula', 'ultimo_ano_renovado', 'codigo_estado_matricula']:
   try:
      empresas_RUES_df2[column] = empresas_RUES_df2[column].astype(int)
      print("convertido a integer: ", column)
      print(empresas_RUES_df2[column].describe())
   except:
      print("error: ", column)
empresas_RUES_df2.describe()

convertido a integer:  codigo_camara
count    4.783254e+06
mean     2.114560e+01
std      1.576164e+01
min      1.000000e+00
25%      5.000000e+00
50%      2.100000e+01
75%      3.200000e+01
max      5.700000e+01
Name: codigo_camara, dtype: float64
convertido a integer:  matricula
count    4.783254e+06
mean     3.184721e+07
std      9.910250e+07
min      1.000000e+00
25%      7.603625e+04
50%      2.144570e+05
75%      3.062153e+06
max      1.310268e+09
Name: matricula, dtype: float64
convertido a integer:  ultimo_ano_renovado
count    4.783254e+06
mean     1.639668e+03
std      7.858114e+02
min      0.000000e+00
25%      2.004000e+03
50%      2.017000e+03
75%      2.022000e+03
max      2.050000e+03
Name: ultimo_ano_renovado, dtype: float64
convertido a integer:  codigo_estado_matricula
count    4.783254e+06
mean     2.271908e+00
std      1.838142e+00
min      1.000000e+00
25%      1.000000e+00
50%      3.000000e+00
75%      3.000000e+00
max      1.700000e+01
Name: codigo_estado_matric

,codigo_camara,matricula,ultimo_ano_renovado,fecha_renovacion,codigo_estado_matricula,codigo_camara_propietario,Codigo_tipo_propietario
count,4.783254e+06,4.783254e+06,4.783254e+06,3.791200e+06,4.783254e+06,4.629421e+06,2.506796e+06
mean,2.114560e+01,3.184721e+07,1.639668e+03,2.016474e+07,2.271908e+00,2.091286e+01,9.302759e-01
std,1.576164e+01,9.910250e+07,7.858114e+02,1.137855e+05,1.838142e+00,1.574085e+01,3.425446e-01
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,5.000000e+00,7.603625e+04,2.004000e+03,2.014033e+07,1.000000e+00,5.000000e+00,1.000000e+00
50%,2.100000e+01,2.144570e+05,2.017000e+03,2.019073e+07,3.000000e+00,2.100000e+01,1.000000e+00
75%,3.200000e+01,3.062153e+06,2.022000e+03,2.022031e+07,3.000000e+00,3.200000e+01,1.000000e+00
max,5.700000e+01,1.310268e+09,2.050000e+03,2.050120e+07,1.700000e+01,9.900000e+01,3.000000e+00


In [ ]:
for column in ['codigo_camara_propietario', 'Codigo_tipo_propietario']:
   try:
      #print(empresas_RUES_df2[column].describe())
      #print(empresas_RUES_df2[column].head())
      # print(pd.to_numeric(empresas_RUES_df2[column]).head())
      empresas_RUES_df2[column] = empresas_RUES_df2[column].astype(str)
      empresas_RUES_df2[column] = empresas_RUES_df2[column].str.replace(".0", "")
      empresas_RUES_df2[column] = empresas_RUES_df2[column].str.replace("nan", "")
      #print(empresas_RUES_df2[column].value_counts())
   except:
      print(column)

In [295]:
#empresas_RUES_df2['fecha_renovacion'] = empresas_RUES_df.loc[empresas_RUES_df2.index]['fecha_renovacion'].copy()
empresas_RUES_df2['fecha_renovacion'] = empresas_RUES_df2['fecha_renovacion'].astype(str)
empresas_RUES_df2['fecha_renovacion'] = empresas_RUES_df2['fecha_renovacion'].str.replace("nan", "").str.replace(".", "" , n=1)
empresas_RUES_df2['fecha_renovacion'] = empresas_RUES_df2['fecha_renovacion'].str.strip().str[:4] + "-" + empresas_RUES_df2['fecha_renovacion'].astype(str).str.strip().str[4:6] + "-" + empresas_RUES_df2['fecha_renovacion'].astype(str).str.strip().str[6:8]
empresas_RUES_df2['fecha_renovacion'].head()

C:\Users\gatew\AppData\Local\Temp\ipykernel_6104\3840038025.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresas_RUES_df2['fecha_renovacion'] = empresas_RUES_df2['fecha_renovacion'].str.replace("nan", "").str.replace(".", "" , n=1)


0    2022-07-22
1    2022-04-25
2    2022-03-29
3    2022-07-11
4            --
Name: fecha_renovacion, dtype: object

In [ ]:
Se debería hacer lo mismo con la fecha_cancelacion?

In [ ]:
# Ver cuantos  registros no tienen bien el digito de Verificacion
#[ NO FUNCIONA ]
#empresas_RUES_df2["digit_ver"] = empresas_RUES_df2["numero_identificacion"].apply(lambda x : calcular_digito_ver(x))
#empresas_RUES_df2[empresas_RUES_df2["digit_ver"] != empresas_RUES_df2["digito_verificacion"]].shape

In [ ]:
#empresas_RUES_df2.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)

### Guardar Datos

In [297]:
# Save the recovered data from the SQL request to the Datos Abiertos API: 
# (Uncomment to save, comment after save in order to not damage the previous file)
empresas_RUES_df2.to_csv('./1.Clean_dataset/empresas_registradas_RUES_clean.csv')

In [148]:
empresas_RUES_df2 = pd.read_csv("./1.Clean_dataset/empresas_registradas_RUES_clean.csv", encoding = 'utf-8', dtype='str', sep=',')
print(empresas_RUES_df2.shape)
empresas_RUES_df2.head()

(4783254, 25)


,Unnamed: 0,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
0,0,23,HUILA,290479,2022,2022-07-22,MANTENIMARMOL,3,CANCELADA,4330,...,CEDULA DE CIUDADANIA,1075287105,00001075287105,0.0,23,HUILA,290478,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
1,1,5,BUCARAMANGA,410242633,2022,2022-04-25,TIENDA ANGELITOS CON AMOR,1,ACTIVA,4759,...,NIT,37896777,00000037896777,3.0,5,BUCARAMANGA,177224,1,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
2,2,45,SUR Y ORIENTE DEL TOLIMA,23329,2022,2022-03-29,BANCO DAVIVIENDA S.A AGENCIA CHAPARRAL,1,ACTIVA,6412,...,NIT,860034313,00000860034313,7.0,4,BOGOTA,276917,NaN,NaN,AGENCIA
3,3,4,BOGOTA,2311772,2022,2022-07-11,ESKALA SHOES,1,ACTIVA,1521,...,CEDULA DE CIUDADANIA,79741513,NaN,NaN,4,BOGOTA,2311768,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
4,4,4,BOGOTA,3559385,0,--,INVICTU'S,1,ACTIVA,4771,...,CEDULA DE CIUDADANIA,1000329396,NaN,NaN,4,BOGOTA,3559382,NaN,NaN,ESTABLECIMIENTO DE COMERCIO


In [298]:
empresas_RUES_df2['estado_matricula'].value_counts().to_frame()

,estado_matricula
CANCELADA,2549359
ACTIVA,2161693
MATRÍCULA CANCELADA LEY 1429,67618
NO ASIGNADO,1910
INACTIVA POR CIERRE DEL ESTABLECIMIENTO,1382
MATRÍCULA CANCELADA POR TRASLADO DE DOMICILIO,844
NO MATRICULADO,262
INACTIVA,95
"MATRÍCULA NUEVA, CONSTITUCIÓN POR TRASLADO",87
MATRICULA INACTIVA POR PERDIDA DE CALIDAD DE COMERCIANTE,4


In [299]:
empresas_RUES_df2['ultimo_ano_renovado'].value_counts().head(10).to_frame()

,ultimo_ano_renovado
2022,1327056
0,893353
2021,310545
2019,274716
2018,219510
2020,201496
2017,169676
2016,157038
2014,136828
2015,132491


## Limpieza de CIIU codes (Actividades Económicas)

In [113]:
PATH = 'CIIU_CLase.csv'
CIIU_codes_df = pd.read_csv(PATH, encoding = 'utf-8', dtype='str', sep=',' )
print(CIIU_codes_df.shape)
CIIU_codes_df

(1058, 2)


,Clase,Descripción
0,111,"Cultivo de cereales (excepto arroz), legumbres..."
1,112,Cultivo de arroz
2,113,"Cultivo de hortalizas, raíces y tubérculos"
3,114,Cultivo de tabaco
4,115,Cultivo de plantas textiles
...,...,...
1053,9999,Actividad No especificada
1054,5211,"Comercio al por menor, en establecimientos no ..."
1055,7110,Actividades económicas de las personas que pre...
1056,5223,"Actividades de aeropuertos, servicios de naveg..."


In [114]:
CIIU_codes_df['Clase'] = CIIU_codes_df['Clase'].apply(limpiar_ciuu_code)
CIIU_codes_df.drop_duplicates(subset="Clase", inplace=True)
CIIU_codes_df.shape

(507, 2)

In [116]:
empresas_RUES_df2['cod_ciiu_act_econ_pri'].unique()

array(['4330', '4759', '6412', ..., '0320', '8200', '5881'], dtype=object)

In [149]:
# Ajustar formato de códigos CIIU con letras y números:
empresas_RUES_df2['cod_ciiu_act_econ_pri'] = empresas_RUES_df2['cod_ciiu_act_econ_pri'].str.extract('(\d+)', expand=False)
empresas_RUES_df2['cod_ciiu_act_econ_sec'] = empresas_RUES_df2['cod_ciiu_act_econ_sec'].str.extract('(\d+)', expand=False)
#empresas_RUES_df['cod_ciiu_act_econ_pri'].value_counts().head(100)

In [150]:
empresas_RUES_df2[(empresas_RUES_df2['cod_ciiu_act_econ_pri'].str.len()<4)]['cod_ciiu_act_econ_pri'].unique()

array(['990', '312', '142', '144', '163', '811', '162', '130', '210',
       '722', '510', '141', '123', '119', '399', '150', '113', '320',
       '820'], dtype=object)

In [151]:
empresas_RUES_df2[(empresas_RUES_df2['cod_ciiu_act_econ_pri'].str.len()!=4)]['cod_ciiu_act_econ_pri'].value_counts()

999999    18920
521102    14834
521100     6311
523300     4049
552100     3777
          ...  
050108        1
292101        1
142104        1
112010        1
605001        1
Name: cod_ciiu_act_econ_pri, Length: 2331, dtype: int64

In [120]:
code_list = empresas_RUES_df2['cod_ciiu_act_econ_pri'].apply(limpiar_ciuu_code).unique()
len(code_list)

1010

In [121]:
len([value for value in code_list if value not in CIIU_codes_df['Clase'].to_list()])

507

In [122]:
series = empresas_RUES_df2['cod_ciiu_act_econ_pri'].copy()
series = series.apply(limpiar_ciuu_code)
#series = series.astype(float).astype(int).astype(str)

In [123]:
print(series.isin(CIIU_codes_df['Clase'].to_list()).shape)
series[series.isin(CIIU_codes_df['Clase'].to_list())].value_counts()

(4783254,)


9999    801577
4711    304542
5611    193636
5630    163831
4771    160145
         ...  
0620         5
6495         4
9492         3
2680         2
6532         1
Name: cod_ciiu_act_econ_pri, Length: 503, dtype: int64

In [124]:
print(series[~series.isin(CIIU_codes_df['Clase'].to_list())].shape)
series[~series.isin(CIIU_codes_df['Clase'].to_list())].value_counts().head(20)

(148856,)


5233    6368
1810    4507
5521    4100
5239    4083
5219    3961
5231    3581
5522    3248
7499    3045
5241    3005
7421    2884
5234    2271
9242    2241
5020    2232
9302    2181
5244    2017
4521    1861
5030    1856
6426    1703
5119    1618
7414    1599
Name: cod_ciiu_act_econ_pri, dtype: int64

In [152]:
# Ajustar formato de códigos CIIU con letras y números:
empresas_RUES_df2['cod_ciiu_act_econ_pri'] = empresas_RUES_df2['cod_ciiu_act_econ_pri'].apply(limpiar_ciuu_code)
empresas_RUES_df2['cod_ciiu_act_econ_sec'] = empresas_RUES_df2['cod_ciiu_act_econ_sec'].apply(limpiar_ciuu_code)

#empresas_RUES_df['cod_ciiu_act_econ_pri'].value_counts().head(100)
empresas_RUES_df2['cod_ciiu_act_econ_pri'].value_counts()

9999    801577
4711    304542
5611    193636
5630    163831
4771    160145
         ...  
3528         1
7511         1
6222         1
1201         1
5881         1
Name: cod_ciiu_act_econ_pri, Length: 1009, dtype: int64

## No aparecen empresas en Bogotá

In [157]:
empresas_RUES_df2['camara_comercio'].value_counts().head(13)

BOGOTA                     572264
MEDELLIN PARA ANTIOQUIA    422411
BUCARAMANGA                303697
CUCUTA                     201417
CALI                       198910
BARRANQUILLA               186542
VILLAVICENCIO              173912
HUILA                      167575
CARTAGENA                  163391
IBAGUE                     136075
PEREIRA                    124347
SANTA MARTA                107797
ARMENIA                    104661
Name: camara_comercio, dtype: int64

In [158]:
empresas_RUES_df2.loc[empresas_RUES_df2['camara_comercio']=='MEDELLIN PARA ANTIOQUIA', 'camara_comercio'] = 'MEDELLIN'

In [159]:
# Por qué no sale ninguna en Bogotá?
# Que tienen en particular los registros que no tienen NIT Propietario? -> que son cédulas
empresas_RUES_df2[(empresas_RUES_df2['camara_comercio']=='BOGOTA') & (empresas_RUES_df2['nit_propietario'].isna())].shape

(474917, 25)

In [160]:
empresas_RUES_df2[(empresas_RUES_df2['camara_comercio']=='BOGOTA') & (empresas_RUES_df2['nit_propietario'].isna()) & (empresas_RUES_df2['codigo_clase_identificacion']!=1)]['clase_identificacion'].value_counts()

CEDULA DE CIUDADANIA            468202
CEDULA DE EXTRANJERIA             3096
PASAPORTE                         2634
DOCUMENTO EXTRANJERO               641
NO DEFINIDO                        225
TARJETA DE IDENTIDAD               103
NIT                                 14
REGISTRO CIVIL DE NACIMIENTO         2
Name: clase_identificacion, dtype: int64

In [ ]:
# Eliminar NITS que no se ha podido conseguir su valor:
# No implementado, se decidió imputar los Nits a partir de los números de identificación disponibles
#empresas_RUES_df2[empresas_RUES_df2["nit_propietario"].isna()]

In [163]:
# Porque no tienen NIT Propietario, entonces hay que ponerles:
#.str.extract('(\d+)', expand=False)
## NO FUNCIONA
#empresas_RUES_df['nit_propietario'] = empresas_RUES_df.apply(lambda row: row['numero_identificacion'].astype(str).apply(extender_nit_string) if row['nit_propietario'] == np.NaN else row['nit_propietario'] , axis=1)
## NO FUNCIONA
#empresas_RUES_df['nit_propietario'].fillna(value=empresas_RUES_df['numero_identificacion'])

# Asignamos el No de Identificación en lugar del NIT. En la mayoría de los casos será el mismo número.
# Usando [:9] se puede asignar unicamente los primeros 9 dígitos, pero no estamos seguros de que que estamos asignando el NIT correcto un numero mayor de casos
# Imputar NITS a partir de los números de identificacion disponibles.
# OJO: al hacer esto se están imputando NITs que puede que no existan.
empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"].isna(), 'nit_propietario'] = empresas_RUES_df2.loc[empresas_RUES_df2["nit_propietario"].isna(), 'numero_identificacion'].apply(lambda x : extender_nit_string(x))
empresas_RUES_df2.isna().sum()

Unnamed: 0                           0
codigo_camara                        0
camara_comercio                      0
matricula                            0
ultimo_ano_renovado                  0
fecha_renovacion                     0
razon_social                        17
codigo_estado_matricula              0
estado_matricula                     0
cod_ciiu_act_econ_pri            24949
cod_ciiu_act_econ_sec          3186915
ciiu3                          4067945
ciiu4                          4459265
fecha_cancelacion              2099733
codigo_clase_identificacion          0
clase_identificacion                 0
numero_identificacion                0
nit_propietario                      0
digito_verificacion            1051166
codigo_camara_propietario       569165
camara_comercio_propitario      153880
matrícula_propietario           122065
Codigo_tipo_propietario        2276458
tipo_propietario               2509369
categoria_matricula                  0
dtype: int64

In [164]:
empresas_RUES_df2.loc[empresas_RUES_df2['nit_propietario'].isnull(), 'nit_propietario'].shape

(0,)

In [165]:
# Se actualiz'o el Dataset?
empresas_RUES_df2[empresas_RUES_df2['camara_comercio']=='BOGOTA'].head()


,Unnamed: 0,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,...,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
3,3,4,BOGOTA,2311772,2022,2022-07-11,ESKALA SHOES,1,ACTIVA,1521,...,CEDULA DE CIUDADANIA,79741513,00000079741513,NaN,4,BOGOTA,2311768,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
4,4,4,BOGOTA,3559385,0,--,INVICTU'S,1,ACTIVA,4771,...,CEDULA DE CIUDADANIA,1000329396,00001000329396,NaN,4,BOGOTA,3559382,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
6,6,4,BOGOTA,1600902,2022,2022-06-28,FRUTOS VICTORIA,1,ACTIVA,4631,...,CEDULA DE CIUDADANIA,79698643,00000079698643,NaN,4,BOGOTA,1600897,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
34,36,4,BOGOTA,2945980,2018,--,ALMOJABANAS JADE,1,ACTIVA,5613,...,CEDULA DE CIUDADANIA,1049618609,00001049618609,0.0,4,BOGOTA,2945979,0,NaN,ESTABLECIMIENTO DE COMERCIO
39,41,4,BOGOTA,3551936,0,--,ALASKA GOURMET,1,ACTIVA,5611,...,CEDULA DE CIUDADANIA,52967774,00000052967774,NaN,4,BOGOTA,3341652,NaN,NaN,ESTABLECIMIENTO DE COMERCIO


In [166]:
empresas_RUES_df2.drop(columns = ['Unnamed: 0', ], inplace=True)

### Guardar Datos

In [168]:
# Save the recovered data from the SQL request to the Datos Abiertos API: 
# (Uncomment to save, comment after save in order to not damage the previous file)
empresas_RUES_df2.to_csv('./1.Clean_dataset/empresas_registradas_RUES_clean_ciiu.csv')

In [ ]:
empresas_RUES_df2 = pd.read_csv("./1.Clean_dataset/empresas_registradas_RUES_clean_ciuu.csv", encoding = 'utf-8', dtype='str', sep=',')
print(empresas_RUES_df2.shape)
empresas_RUES_df2.head()